In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
from skimage.filters import gaussian, sobel, scharr
from skimage.segmentation import active_contour
from skimage.feature import canny
from skimage.exposure import histogram, equalize_hist
from scipy import ndimage as ndi
from skimage.color import label2rgb
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [13]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, 
                          fours, fulls, ratios, inc): 
    
   
    frratios, flratios = ratios
    #print(frratios, flratios)
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = morphology.binary_opening(thresh_img, disk(8))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs =skimage.feature.blob_doh(img, min_sigma =8, max_sigma = 17, threshold = 0.007, num_sigma= 15, overlap=0.6)
        
    ghighlight = gaussian(
        morphology.binary_opening(
             (thresh_img^phighlight)
                           , square(4)), sigma = 0.65)
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =skimage.feature.blob_doh(img, min_sigma =9, max_sigma = 16, threshold = 0.005, num_sigma= 15, overlap=0.2)
    

    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
            
    plas_on_glas_dists =[connect((pblob[1], pblob[0]),m_bond, gblobs) for pblob in pblobs]
    num_mconnections = reducer(plas_on_glas_dists)


    glas_on_glas_dists =[connect((gblobs[i][1], gblobs[i][0]),g_bond, gblobs[i+1:]) for i in range(ngblobs-1)]
    num_gconnections = reducer(glas_on_glas_dists)


    plas_on_plas_dists = [connect((pblobs[i][1], pblobs[i][0]),p_bond, pblobs[i+1:]) for i in range(npblobs-1)]
    num_pconnections = reducer(plas_on_plas_dists)


    num_connections = sum([num_pconnections, num_gconnections,num_mconnections])
    #print(num_connections)
    
    outline_sep = scharr(frame)
    markers = np.zeros_like(frame)
    markers[frame < threshold_mean(frame)] = 1
    markers[frame > threshold_triangle(frame)+2] = 2
    segmentation = morphology.watershed(outline_sep, markers)
    segmentation = morphology.opening(ndi.binary_fill_holes(segmentation - 1), disk(3))
    labeled_balls_sep, _ = ndi.label(segmentation)
    regions = measure.regionprops(labeled_balls_sep)

    
    #----------------------filter------------------------- can use actual filter
    if num_connections ==0 and num_blobs == 4:
        fours += inc
                
        plasareas = areas(regions, pblobs) 
        glasareas = areas(regions, gblobs)
        garea = average(glasareas)
        plarea = average(plasareas)
        frratios.append(plarea/garea)
        #print(frratios)
    elif  num_connections >=4 and num_blobs == 4: 
        fulls += inc
             
        imtog = np.copy(frame)
        highs = imtog > threshold_minimum(imtog)
        highs= morphology.opening(highs, disk(8))
        imtog[highs==0]=0
        img = scharr(imtog)
        markers = np.zeros_like(frame)
        markers[imtog < threshold_mean(frame)] = 1
        markers[imtog > threshold_minimum(frame)+2] = 2
        segmentation = morphology.watershed(img, markers)
        segmentation = ndi.binary_fill_holes(morphology.closing(segmentation - 1, disk(5)))
        labeled_balls_tog, _ = ndi.label(segmentation)
        region=measure.regionprops(labeled_balls_tog)
        

        allarea = areafinder(regions)
        plarea = areafinder(region)/npblobs
        flratios.append(3*plarea/(allarea-plarea))
        #print(flratios)    
            
    else:
        pass
    #print(ratios)
    return [fours, fulls, ratios]



        
# take largest isodata image region, and return aclassification
def areas(regions, blobs):    
    # mathc blob o blob, collect areas into a list
    areas = []
    for blob in blobs:
        for reg in regions:
            a = reg.area
            coords = reg.centroid
            if coords[0] == blob[0] and coords[1]==blobs[1]:
                areas.append(a)
                break
    return(areas)

def areafinder(allregions):    
    # mathc blob o blob, collect areas into a list
    #print(allregions[0].area)
    aregion = max(allregions, key=attrgetter('area'))
    return(aregion.area)
def reducer(connections):
    return sum(connections)    
def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    norm_radius = 15
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        if vector_d <= 2*radius and vector_d > .33*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

In [ ]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/W*half/*_21_a.avi")#glob("./tchakamau/W*half/*_[4,7,8]*.avi") +glob("./tchakamau/W*half/*a.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)

trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.35
g_bond = 15*1.15
m_bond = 15*1.1
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, g_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
limit = 2
frratio = []
flratio = []
ratios = [frratio, flratio]
        
for vidnum in range(len(filenames)):
    print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
    #meta = ffprobe(filenames[vidnum])
    #print(meta)#['@nb_frames'])
    #print(5)
    frames = vreader(filenames[vidnum])
    #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    #print(frames)



    sides = {"p":[], "g":[], "b":[], "ucf":[]}
    classes = {"p":[], "g":[], "b":[], "ucf":[]}
    transitions = [("p->p",[]),("p->g",[]), ("p->b",[]),
                   ("g->p",[]),("g->g",[]), ("g->b",[]),
                  ("b->p",[]),("b->g",[]), ("b->b",[])]

    transitions = OrderedDict(transitions)
    broken_count = 0
    class_num = 0
    origin = 'ucf'
    last_class = 'ucf'
    last_whole = 0
    fours = 0
    fulls = 0
    fnum = 0
    inc = 20
    for fr in frames:
        frame = fr[:, :, 2]
        org = total_threshold_filter(frame, fours, fulls, ratios, inc)
        fours, fulls, ratios = org
        #print(fours, fulls)
        if fours > limit and fulls > limit:
            break
    print(fours, fulls, ratios)

    index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------



['./tchakamau\\Washed15vhalf\\45782_15Vpp_21_a.avi']
Number files:  1
Processing vid 0 : ./tchakamau\Washed15vhalf\45782_15Vpp_21_a.avi
